In [22]:
import pyarrow.parquet as pq
import pandas as pd
import os
import googlemaps

In [118]:
GOOGLE_MAPS_API_KEY = os.environ['GOOGLE_MAPS_API_KEY']
gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

# Get intervention coordinates

In [2]:
file_configs = {
        'interventions3.parquet.gzip': {
            'event_types': [
                "P003 - Cardiac arrest",
                "P014 - Electrocution - electrification",
                "P019 - Unconscious - syncope",
                "P011 - Chest pain"
            ],
            'columns': {
                'longitude': "Longitude intervention",
                'latitude': "Latitude intervention",
                'event': "EventType Trip"
            }
        },
        'interventions1.parquet.gzip': {
            'event_types': [
                "P003 - Cardiac arrest",
                "P014 - Electrocution - electrification",
                "P019 - Unconscious - syncope",
                "P011 - Chest pain"
            ],
            'columns': {
                'longitude': "Longitude intervention",
                'latitude': "Latitude intervention",
                'event': "EventType Trip"
            }
        },
        'interventions_bxl2.parquet.gzip': {
            'event_types': [
                'HARTSTILSTAND - DOOD - OVERLEDEN',
                'PIJN OP DE BORST',
                'CARDIAAL PROBLEEM (ANDERE DAN PIJN AAN DE BORST)'
            ],
            'columns': {
                'longitude': "Longitude intervention",
                'latitude': "Latitude intervention",
                'event': "EventType and EventLevel"
            }
        },
        'interventions_bxl.parquet.gzip': {
            'event_types': [
                'P003 - Cardiac arrest',
                'P019 - Unconscious - syncope',
                'P011 - Chest pain',
                'P029 - Obstruction of the respiratory tract',
                'P014 - Electrocution - electrification',
                'TI (3.3.1) rescue electrocution/electrification'
            ],
            'columns': {
                'longitude': "longitude_intervention",
                'latitude': "latitude_intervention",
                'event': "eventtype_trip"
            }
        }
    }

In [13]:
def format_coordinates(longitude, latitude):
    formatted_longitude = str(longitude)[:1] + '.' + str(longitude).replace('.', '')[1:]
    formatted_latitude = str(latitude)[:2] + '.' + str(latitude).replace('.', '')[2:]
    return formatted_longitude, formatted_latitude


def process_and_display_map_data(df, selected_file, show_cardiac_incidences):
    map_data = pd.DataFrame(columns=['lat', 'lon'])
    config = file_configs[selected_file]

    for index, row in df.iterrows():
        current_longitude = row[config['columns']['longitude']]
        current_latitude = row[config['columns']['latitude']]
        if pd.isnull(current_longitude) or pd.isnull(current_latitude):
            continue

        current_longitude, current_latitude = format_coordinates(current_longitude, current_latitude)

        if show_cardiac_incidences == 'Yes':
            is_interesting = any(event_type in row[config['columns']['event']] for event_type in config['event_types']) if row[config['columns']['event']] else False
            if is_interesting:
                map_data = pd.concat([map_data, pd.DataFrame({'lat': [float(current_latitude)], 'lon': [float(current_longitude)]})], ignore_index=True)
        else:
            map_data = pd.concat([map_data, pd.DataFrame({'lat': [float(current_latitude)], 'lon': [float(current_longitude)]})], ignore_index=True)
    
    return map_data

In [10]:
intervention1 = pq.ParquetFile(os.path.join('../data', 'interventions1.parquet.gzip')).read().to_pandas()
intervention2 = pq.ParquetFile(os.path.join('../data', 'interventions2.parquet.gzip')).read().to_pandas()
intervention3 = pq.ParquetFile(os.path.join('../data', 'interventions3.parquet.gzip')).read().to_pandas()

In [16]:
coordinates1 = process_and_display_map_data(intervention1, 'interventions1.parquet.gzip', show_cardiac_incidences='Yes')
coordinates2 = process_and_display_map_data(intervention2, 'interventions1.parquet.gzip', show_cardiac_incidences='Yes')
coordinates3 = process_and_display_map_data(intervention3, 'interventions1.parquet.gzip', show_cardiac_incidences='Yes')

/var/folders/8t/4mtztt010752cj_569fwq_h00000gp/T/ipykernel_76561/3342450.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  map_data = pd.concat([map_data, pd.DataFrame({'lat': [float(current_latitude)], 'lon': [float(current_longitude)]})], ignore_index=True)
/var/folders/8t/4mtztt010752cj_569fwq_h00000gp/T/ipykernel_76561/3342450.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  map_data = pd.concat([map_data, pd.DataFrame({'lat': [float(current_latitude)], 'lon': [float(current_longitude

In [95]:
intervention_coordinates = pd.concat([coordinates1, coordinates2, coordinates3], ignore_index=True)
intervention_coordinates = intervention_coordinates.drop_duplicates()
intervention_coordinates['id'] = range(len(intervention_coordinates))
intervention_coordinates.to_csv('../data/intervention_coordinates.csv', index=False)

# Get AED coordinates

In [62]:
aed_locations = pq.ParquetFile(os.path.join('../data', 'aed_locations.parquet.gzip')).read().to_pandas()

In [ ]:
aed_addresses = []
for index, location in aed_locations.iterrows():
    number = '' if pd.isnull(location['number']) else location['number']
    postal_code = '' if pd.isnull(location['postal_code']) else int(location['postal_code'])
    address = f"{number} {location['address']}, {location['municipality']}, {location['province']}, Belgium {postal_code}"
    aed_addresses.append(address)

aed_locations['full_address'] = aed_addresses

In [97]:
aed_locations = aed_locations.drop_duplicates(subset=['full_address'])
aed_locations['id'] = range(len(aed_locations))
aed_locations

,id,type,address,number,postal_code,municipality,province,location,public,available,hours,full_address
0,0,None,Blvd. Fr. Roosevelt,24.0,7060.0,SOIGNIES,Hainaut,None,Y,None,None,"24.0 Blvd. Fr. Roosevelt, SOIGNIES, Hainaut, B..."
1,1,None,Ch. De Wégimont,76.0,4630.0,Ayeneux,Liège,None,None,None,None,"76.0 Ch. De Wégimont, Ayeneux, Liège, Belgium ..."
2,2,None,Place Saint - Lambert,NaN,4020.0,Liège,Liège,None,None,None,None,"Place Saint - Lambert, Liège, Liège, Belgium ..."
3,3,None,Rue du Doyard,NaN,4990.0,Lierneux,Liège,None,None,None,None,"Rue du Doyard, Lierneux, Liège, Belgium 4990"
4,4,None,Fond Saint Servais,NaN,4000.0,Liège,Liège,None,None,None,None,"Fond Saint Servais, Liège, Liège, Belgium 4000"
...,...,...,...,...,...,...,...,...,...,...,...,...
15221,13799,Appareil fixe-Vast apparaat,Nekkerspoel-Borcht,19.0,2800.0,Mechelen,Antwerpen,"Hoofdgebouw, 1ste verdieping, thv cafetaria",Oui-Ja,Non-Nee,maandag tot en met vrijdag van 8 tot 23 uur; w...,"19.0 Nekkerspoel-Borcht , Mechelen, Antwerpen,..."
15223,13800,Appareil fixe-Vast apparaat,Nieuwe Dreef,17.0,9160.0,Lokeren,Oost-Vlaanderen,Locatie : ingang poort hoofdgebouw,Oui-Ja,Oui-Ja,None,"17.0 Nieuwe Dreef , Lokeren, Oost-Vlaanderen, ..."
15224,13801,Appareil fixe-Vast apparaat,Panterschipstraat,207.0,9000.0,Gent,Oost-Vlaanderen,,Oui-Ja,Non-Nee,None,"207.0 Panterschipstraat , Gent, Oost-Vlaandere..."
15225,13802,Appareil fixe-Vast apparaat,Albert Leemansplein,20.0,1050.0,Bruxelles,Bruxelles-Brussel,,Oui-Ja,Non-Nee,None,"20.0 Albert Leemansplein, Bruxelles, Bruxelles..."


In [101]:
def get_lat_lon(address):
    # Geocode an address
    result = gmaps.geocode(address)

    # If result is not empty, extract latitude and longitude
    if result:
        location = result[0]['geometry']['location']
        return location['lat'], location['lng']
    else:
        return None, None

In [105]:
aed_lats = []
aed_lons = []

for address in aed_locations['full_address']:
    lat, lon = get_lat_lon(address)
    aed_lats.append(lat)
    aed_lons.append(lon)

In [108]:
aed_locations['lat'] = aed_lats
aed_locations['lon'] = aed_lons

In [110]:
aed_locations.to_csv('../data/aed_locations_full.csv', index=False)

# Get distance between intervention & AED coordinates